In [ ]:
!pip install -U transformers

In [ ]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torch
from PIL import Image
import pandas as pd
import os

print("Loading Emiel baseline model...")
model_name = "Emiel/cub-200-bird-classifier-swin"
processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModelForImageClassification.from_pretrained(model_name)
model.eval()

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
print(f"Using device: {device}")

test_df = pd.read_csv('/kaggle/input/feathers-in-focus/test_images_path.csv')
print(f"Number of test images: {len(test_df)}")

print("\nPredicting test set...")
predictions = []
for idx, row in test_df.iterrows():
    filename = row['image_path'].split('/')[-1]
    img_path = f"/kaggle/input/feathers-in-focus/test_images/test_images/{filename}"
    
    image = Image.open(img_path).convert('RGB')
    inputs = processor(images=image, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_class = outputs.logits.argmax(-1).item()
    
    predictions.append(predicted_class + 1)
    
    if (idx + 1) % 500 == 0:
        print(f"Processed {idx + 1}/{len(test_df)} images")

submission = pd.DataFrame({
    'id': test_df['id'],
    'label': predictions
})

submission.to_csv('submission.csv', index=False)
print("\n✓ Test predictions saved to submission.csv")
print(submission.head(10))

print("\n\n" + "="*60)
print("EVALUATING ON TRAIN SET (to estimate performance)")
print("="*60)

train_df = pd.read_csv('/kaggle/input/feathers-in-focus/train_images.csv')
train_sample = train_df.sample(n=500, random_state=42)
print(f"Evaluating on {len(train_sample)} train images...")

correct = 0
total = 0

for idx, row in train_sample.iterrows():
    filename = row['image_path'].split('/')[-1]
    img_path = f"/kaggle/input/feathers-in-focus/train_images/train_images/{filename}"
    true_label = row['label'] - 1
    
    try:
        image = Image.open(img_path).convert('RGB')
        inputs = processor(images=image, return_tensors="pt").to(device)
        
        with torch.no_grad():
            outputs = model(**inputs)
            predicted_class = outputs.logits.argmax(-1).item()
        
        if predicted_class == true_label:
            correct += 1
        total += 1
        
        if total % 100 == 0:
            print(f"Evaluated {total}/{len(train_sample)} images")
    except Exception as e:
        print(f"Error on image {filename}: {e}")
        continue

accuracy = correct / total
print(f"\n{'='*60}")
print(f"BASELINE PERFORMANCE ON TRAIN SAMPLE:")
print(f"Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Correct: {correct}/{total}")
print(f"{'='*60}")

print("\n\n" + "="*60)
print("SUBMISSION FILE VERIFICATION")
print("="*60)

if os.path.exists('/kaggle/working/submission.csv'):
    print("✓ submission.csv exists!")
    check_df = pd.read_csv('/kaggle/working/submission.csv')
    print(f"✓ Number of rows: {len(check_df)}")
    print(f"✓ Columns: {check_df.columns.tolist()}")
    print(f"✓ Label range: {check_df['label'].min()} - {check_df['label'].max()}")
    print(f"\nFirst 10 rows:")
    print(check_df.head(10))
    
    if check_df['label'].min() >= 1 and check_df['label'].max() <= 200:
        print("\n✅ Labels are correct (1-200)! Ready to submit!")
    else:
        print("\n⚠️ WARNING: Labels out of range!")
else:
    print("❌ submission.csv NOT found!")

In [ ]:
import os
import time

file_path = '/kaggle/working/submission.csv'
if os.path.exists(file_path):
    timestamp = os.path.getmtime(file_path)
    readable_time = time.ctime(timestamp)
    print(f"File last modified: {readable_time}")
else:
    print("File doesn't exist")

In [ ]:
from datetime import datetime
import os

file_path = '/kaggle/working/submission.csv'
if os.path.exists(file_path):
    # Server tijd (UTC)
    mod_time = os.path.getmtime(file_path)
    file_time = datetime.fromtimestamp(mod_time)
    current_time = datetime.now()
    
    print(f"File modified: {file_time.strftime('%H:%M:%S')}")
    print(f"Current time: {current_time.strftime('%H:%M:%S')}")
    print(f"Difference: {(current_time - file_time).seconds} seconds ago")
    
    if (current_time - file_time).seconds < 600:  # Minder dan 10 minuten
        print("\n✅ File is FRESH (made in last 10 minutes)!")
    else:
        print("\n⚠️ File is OLD")

## Local Inference on GPU 
Model page: https://huggingface.co/Emiel/cub-200-bird-classifier-swin

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/Emiel/cub-200-bird-classifier-swin)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("image-classification", model="Emiel/cub-200-bird-classifier-swin")
pipe("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/hub/parrots.png")

In [ ]:
# Load model directly
from transformers import AutoImageProcessor, AutoModelForImageClassification

processor = AutoImageProcessor.from_pretrained("Emiel/cub-200-bird-classifier-swin")
model = AutoModelForImageClassification.from_pretrained("Emiel/cub-200-bird-classifier-swin")